In [43]:
import numpy as np

In [44]:
M = 20  # kg
g = 9.8 # m / s^2
m = 0.5 # kg
l = 0.5 # m

In [45]:
v1 = ((M + m) * g) / (M * l)
v2 = -(m * g) / M

c1 = -1 / (M * l)
c2 = 1 / M

In [46]:
A = np.array([[0, 1, 0, 0], [0, 0, v2, 0], [0, 0, 0, 1], [0, 0, v1, 0]])
B = np.array([[0], [c2], [0], [c1]])

In [47]:
print(np.shape(A))
print(np.shape(B))

(4, 4)
(4, 1)


## Controllability

CM = [B AB A<sup>2</sup>B A<sup>3</sup>B]

In [48]:
CM = np.array([[B], [A @ B], [A @ A @ B], [A @ A @ A @ B]]).reshape(4, 4)

In [49]:
print((CM))

[[ 0.      0.05    0.     -0.1   ]
 [ 0.05    0.     -0.1     0.    ]
 [ 0.      0.0245  0.     -2.009 ]
 [ 0.0245  0.     -2.009   0.    ]]


In [50]:
rank = np.linalg.matrix_rank(CM)
print("Rank of CM: ", rank)

Rank of CM:  4


> Rank of CM = 4, therefore controllable

## Stability


In [51]:
print(A)

[[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]


In [52]:
eigenvalues = np.linalg.eigvals(A)

print("Eigenvalues of matrix A:", eigenvalues)

Eigenvalues of matrix A: [ 0.          0.          4.48218697 -4.48218697]


>One eigen value is positive, so the system is unstable

## Policy Iteration


In [53]:
Q = np.diag([100, 100, 1, 1])
R = np.array([[0.1]])


In [54]:
print("A:\n", A)
print("B:\n", B)

A:
 [[ 0.     1.     0.     0.   ]
 [ 0.     0.    -0.245  0.   ]
 [ 0.     0.     0.     1.   ]
 [ 0.     0.    20.09   0.   ]]
B:
 [[ 0.  ]
 [ 0.05]
 [ 0.  ]
 [-0.1 ]]


In [55]:
F = [[ 0.        ],
 [ 0.01276981],
 [ 0.        ],
 [-0.02003184]]

F = np.array(F).reshape(1, 4)
print("Eigenvalues of A + BF:", np.linalg.eigvals(A + B @ F))

Eigenvalues of A + BF: [ 0.00000000e+00  6.22917522e-04  4.48319660e+00 -4.48117784e+00]


In [56]:
import control
# Define the desired closed-loop poles
desired_poles = np.array([-20, 10, -3, -2])

# Compute the feedback gain matrix F that places the closed-loop poles at the desired locations
F = control.place(A, B, desired_poles)

# Check if the closed-loop system is stable
if np.all(np.real(np.linalg.eigvals(A + B @ F)) < 0):
    print("Closed-loop system is stable")
else:
    print("Closed-loop system is unstable")


Closed-loop system is unstable


## Hewer's Algorithm

In [57]:
def policy_iteration(A, B, Q, R, F):
    # P is I in 4x4
    P = np.identity(4)

    
    for i in range(100):
        F = -np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A
        P = Q + F.T @ R @ F + (A + B @ F).T @ P @ (A + B @ F)

    return F

 
F = policy_iteration(A, B, Q, R, F)

print("F", F)

F [[  0.           0.         200.39200062   0.        ]]


In [58]:
import cv2

class InvertedPendulum:
    def __init__(self):
        f = 0

    def step( self, state_vec, t=None ):
        """ state vector :
                x0 : position of the cart
                x1 : veclocity of the cart
                x2 : angle of pendulum. In ref frame with x as forward of the cart and y as up. Angile with respect to ground plane
                x3 : angular velocity of the pendulum
        """
        CART_POS = state_vec[0]
        BOB_ANG  = state_vec[2]*180. / np.pi  # degrees
        LENGTH_OF_PENDULUM = 110.

        IM = np.zeros( (512, 512,3), dtype='uint8' )
        # IM = cv2.imread('imgs/background.jpg').resize((512, 512))

        # Ground line
        cv2.line(IM, (0, 450), (IM.shape[1], 450), (100,20,40), 4 )


        # Mark ground line
        XSTART = -5.
        XEND = 30.
        for xd in np.linspace( XSTART, XEND, 11 ):
            x = int(   (xd - XSTART) / (XEND - XSTART) * IM.shape[0]   )

            cv2.circle( IM, (x, 450), 5, (0,0,255), -1 )

            cv2.putText(IM, str(xd), (x-15,450+15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (100,255,10), 1)


        # Draw Wheels of the cart
        wheel_1_pos = int(   (CART_POS - 2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        wheel_2_pos = int(   (CART_POS + 2 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.circle( IM, (wheel_1_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_2_pos, 415), 25, (255,255,255), 6 )
        cv2.circle( IM, (wheel_1_pos, 415), 2, (255,255,255), -1 )
        cv2.circle( IM, (wheel_2_pos, 415), 2, (255,255,255), -1 )

        # Cart base
        cart_base_start = int(   (CART_POS - 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        cart_base_end   = int(   (CART_POS + 2.5 - XSTART) / (XEND - XSTART) * IM.shape[0]   )

        cv2.line( IM, (cart_base_start, 380), (cart_base_end, 380), (100,100,100), 6 )

        # Pendulum hinge
        pendulum_hinge_x = int(   (CART_POS - XSTART) / (XEND - XSTART) * IM.shape[0]   )
        pendulum_hinge_y = 380
        cv2.circle( IM, (pendulum_hinge_x, pendulum_hinge_y), 10, (100,100,100), -1 )


        # Pendulum
        pendulum_bob_x = int( LENGTH_OF_PENDULUM * np.cos( BOB_ANG / 180. * np.pi ) )
        pendulum_bob_y = int( LENGTH_OF_PENDULUM * np.sin( BOB_ANG / 180. * np.pi ) )
        cv2.circle( IM, (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), 10, (255,255,255), -1 )
        cv2.line( IM, (pendulum_hinge_x, pendulum_hinge_y), (pendulum_hinge_x+pendulum_bob_x, pendulum_hinge_y-pendulum_bob_y), (255,255,255), 3 )

        # Mark the current angle
        angle_display = BOB_ANG % 360
        if( angle_display > 180 ):
            angle_display = -360+angle_display
        cv2.putText(IM, "theta="+str( np.round(angle_display,4) )+" deg", (pendulum_hinge_x-15, pendulum_hinge_y-15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,250), 1)


        # Display on top
        if t is not None:
            cv2.putText(IM, "t="+str(np.round(t,4))+"sec", (15, 15), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)
            cv2.putText(IM, "ANG="+str(np.round(BOB_ANG,4))+" degrees", (15, 35), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)
            cv2.putText(IM, "POS="+str(np.round(CART_POS,4))+" m", (15, 55), cv2.FONT_HERSHEY_COMPLEX_SMALL, 0.8, (200,200,0), 1)

        return IM

In [62]:
sys = InvertedPendulum()

# Initial state
x0 = np.array([[0], [0], [np.pi/6], [0.]])

# U = F @ x, loop and store U and x
x = x0
U = []
X = []

for i in range(1000):
    u = F @ x
    x_dot = A @ x + B @ u
    
    x = x + x_dot * 0.01

    U.append(u)
    X.append(x)

In [63]:
# print eig values of A + BF
print("Eigen values of A + BF", np.linalg.eigvals(A + B @ F))

Eigen values of A + BF [ 0.          0.          0.22538842 -0.22538842]


In [64]:
syst = InvertedPendulum()

for i in range(len(X)):
    rendered = syst.step( [X[i][0], X[i][1], X[i][2], X[i][3]], t=i*0.01)
    print(X[i][2])
    cv2.imshow( 'im', rendered )

    # Press q to exit

    if cv2.waitKey(5) == ord('q'):
        break

[0.52359878]
[0.52360144]
[0.52360676]
[0.52361473]
[0.52362537]
[0.52363867]
[0.52365463]
[0.52367325]
[0.52369453]
[0.52371847]
[0.52374507]
[0.52377433]
[0.52380626]
[0.52384084]
[0.52387808]
[0.52391799]
[0.52396055]
[0.52400578]
[0.52405367]
[0.52410422]
[0.52415743]
[0.52421331]
[0.52427184]
[0.52433305]
[0.52439691]
[0.52446344]
[0.52453263]
[0.52460449]
[0.52467901]
[0.52475619]
[0.52483604]
[0.52491856]
[0.52500374]
[0.52509159]
[0.52518211]
[0.5252753]
[0.52537115]
[0.52546967]
[0.52557086]
[0.52567472]
[0.52578124]
[0.52589044]
[0.52600231]
[0.52611686]
[0.52623407]
[0.52635396]
[0.52647652]
[0.52660175]
[0.52672966]
[0.52686024]
[0.5269935]
[0.52712943]
[0.52726805]
[0.52740934]
[0.52755331]
[0.52769995]
[0.52784928]
[0.52800129]
[0.52815598]
[0.52831335]
[0.52847341]
[0.52863615]
[0.52880157]
[0.52896968]
[0.52914048]
[0.52931396]
[0.52949013]
[0.52966899]
[0.52985054]
[0.53003478]
[0.53022171]
[0.53041134]
[0.53060366]
[0.53079867]
[0.53099638]
[0.53119678]
[0.53139988]
[

KeyboardInterrupt: 